In [3]:
# 1. Clone the official Ultralytics (YOLOv8) repository
!git clone https://github.com/ultralytics/ultralytics

# 2. Navigate into the cloned directory
%cd ultralytics

# 3. Install the required dependencies
!pip install -e .

Cloning into 'ultralytics'...
remote: Enumerating objects: 79689, done.
remote: Counting objects: 100% (849/849), done.
remote: Compressing objects: 100% (472/472), done.
remote: Total 79689 (delta 674), reused 378 (delta 377), pack-reused 78840 (from 4)
Receiving objects: 100% (79689/79689), 43.14 MiB | 19.08 MiB/s, done.
Resolving deltas: 100% (59756/59756), done.
/content/ultralytics/ultralytics/ultralytics
Obtaining file:///content/ultralytics/ultralytics/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.3.252-0.editable-py3-none-any.whl size=23182 sha256=3a8bdb78a4fdb47e669c6fa7ba3622f81a3768dbc450d035e025b16c942bd7f0
  Stored in directory: /tmp/pip-ephem-wheel-cache-u5k2jupy/wheels/49/90/00/6a9dae

In [4]:
# Verify the installation
!yolo checks

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.7/112.6 GB disk)

OS                     Linux-6.6.105+-x86_64-with-glibc2.35
Environment            Colab
Python                 3.12.12
Install                git
Path                   /content/ultralytics/ultralytics/ultralytics/ultralytics
RAM                    12.67 GB
Disk                   39.7/112.6 GB
CPU                    Intel Xeon CPU @ 2.00GHz
CPU count              2
GPU                    Tesla T4, 15095MiB
GPU count              1
CUDA                   12.6

numpy                  ✅ 2.0.2>=1.23.0
matplotlib             ✅ 3.10.0>=3.3.0
ope

In [5]:
# 1. Unzip the specific file to the 'datasets' folder
!unzip -q "/content/Multi-hazard_Labeled_Dataset.zip" -d "/content/datasets"

# 2. Check the contents to see where the images landed
!ls -R /content/datasets | head -n 20

/content/datasets:
Multi-hazard_Labeled_Dataset

/content/datasets/Multi-hazard_Labeled_Dataset:
images
labels

/content/datasets/Multi-hazard_Labeled_Dataset/images:
test
train

/content/datasets/Multi-hazard_Labeled_Dataset/images/test:
crosswalk
pothole
speed_bump

/content/datasets/Multi-hazard_Labeled_Dataset/images/test/crosswalk:
Screenshot 2026-01-06 at 1.38.57 PM.png
Screenshot 2026-01-06 at 1.52.33 PM.png
Screenshot 2026-01-06 at 1.53.03 PM.png


In [6]:
# Run inference using the official YOLO11 Nano model
# This auto-downloads 'yolo11n.pt' and the sample image
!yolo predict model=yolo11n.pt source='https://ultralytics.com/images/bus.jpg'

Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

image 1/1 /content/ultralytics/ultralytics/ultralytics/bus.jpg: 640x480 4 persons, 1 bus, 48.3ms
Speed: 12.3ms preprocess, 48.3ms inference, 36.8ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /content/ultralytics/ultralytics/ultralytics/runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [7]:
import os

# Define the dataset root path based on where you unzipped the file
dataset_root = '/content/datasets/Multi-hazard_Labeled_Dataset'

# Create the YAML content
yaml_content = f"""
path: {dataset_root}  # Dataset root dir
train: images/train   # Train images (YOLO recursively finds images in subfolders)
val: images/test      # Using your 'test' folder as validation

# Classes
nc: 3  # number of classes
names: ['Pothole', 'Speed Bump', 'Crosswalk']
"""

# Write the file to the current working directory
with open('multi_hazard.yaml', 'w') as f:
    f.write(yaml_content)

print(f"✅ 'multi_hazard.yaml' created successfully!")
print(f"   Pointing to: {dataset_root}")

✅ 'multi_hazard.yaml' created successfully!
   Pointing to: /content/datasets/Multi-hazard_Labeled_Dataset


In [8]:
!yolo detect train \
model=yolo11n.pt \
data=multi_hazard.yaml \
epochs=100 \
imgsz=640 \
batch=16 \
cos_lr=True \
close_mosaic=10 \
project=road_hazard_project \
name=exp_best_hyperparams

Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=multi_hazard.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp_best_hyperparams, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, po

In [13]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [14]:
import os

# Define the YouTube URLs for each hazard type
# You can swap these URLs if you find better ones!
videos = {
    "pothole": "https://www.youtube.com/watch?v=LAMSXO1C2z4",    # Pothole dashcam
    "speed_bump": "https://www.youtube.com/watch?v=gm5EY6nnW4k", # Speed Bumps
    "crosswalk": "https://www.youtube.com/watch?v=i8KrlhPPGGo"   # Crosswalks
}

# Loop through and process each video
for i, (hazard, url) in enumerate(videos.items(), start=1):
    print(f"\n⬇️ Downloading {hazard} video...")

    # 1. Download video using yt-dlp and rename it safely
    # -f "b[ext=mp4]" gets the best quality mp4
    filename = f"{hazard}_test.mp4"
    !yt-dlp -f "b[ext=mp4]" --force-overwrites -o "{filename}" "{url}"

    # 2. Run YOLO Prediction
    # We set name=predict1, predict2, etc. to match your request
    print(f"🚀 Running YOLO inference for {hazard}...")

    !yolo predict \
    model=road_hazard_project/exp_best_hyperparams/weights/best.pt \
    source="{filename}" \
    save=True \
    project=ultralytics/runs/detect \
    name=predict{i}

print("\n✅ All testing complete!")
print("📁 Check your results here:")
print("   - Pothole: ultralytics/runs/detect/predict1")
print("   - Speed Bump: ultralytics/runs/detect/predict2")
print("   - Crosswalk: ultralytics/runs/detect/predict3")

Streaming output truncated to the last 5000 lines.
video 1/1 (frame 15250/20240) /content/ultralytics/ultralytics/ultralytics/crosswalk_test.mp4: 384x640 (no detections), 9.5ms
video 1/1 (frame 15251/20240) /content/ultralytics/ultralytics/ultralytics/crosswalk_test.mp4: 384x640 (no detections), 9.4ms
video 1/1 (frame 15252/20240) /content/ultralytics/ultralytics/ultralytics/crosswalk_test.mp4: 384x640 (no detections), 9.7ms
video 1/1 (frame 15253/20240) /content/ultralytics/ultralytics/ultralytics/crosswalk_test.mp4: 384x640 (no detections), 10.0ms
video 1/1 (frame 15254/20240) /content/ultralytics/ultralytics/ultralytics/crosswalk_test.mp4: 384x640 (no detections), 12.1ms
video 1/1 (frame 15255/20240) /content/ultralytics/ultralytics/ultralytics/crosswalk_test.mp4: 384x640 (no detections), 7.5ms
video 1/1 (frame 15256/20240) /content/ultralytics/ultralytics/ultralytics/crosswalk_test.mp4: 384x640 (no detections), 9.1ms
video 1/1 (frame 15257/20240) /content/ultralytics/ultralytics/ul

In [10]:
import os
import glob

# 1. Find a real image file inside your test folder (looking into subfolders)
test_path = '/content/datasets/Multi-hazard_Labeled_Dataset/images/test'
found_images = glob.glob(os.path.join(test_path, '**', '*.jpg'), recursive=True)

if not found_images:
    # Try png if jpg not found
    found_images = glob.glob(os.path.join(test_path, '**', '*.png'), recursive=True)

if found_images:
    # Pick the first image found
    sample_image = found_images[0]
    print(f"✅ Found an image to test: {sample_image}")

    # 2. Run YOLO Prediction on THIS specific image
    # Note: We use !python -m ultralytics to be safe, or just !yolo
    print("🚀 Running prediction...")
    !yolo predict \
    model=road_hazard_project/exp_best_hyperparams/weights/best.pt \
    source="{sample_image}" \
    conf=0.25 \
    save=True \
    project=road_hazard_project \
    name=inference_results

else:
    print("❌ ERROR: Still cannot find any images in 'images/test'.")
    print("Please check if your dataset unzip command actually worked.")
    print("Try running: !ls -R /content/datasets/Multi-hazard_Labeled_Dataset/images/test")

✅ Found an image to test: /content/datasets/Multi-hazard_Labeled_Dataset/images/test/speed_bump/Image00085.jpg
🚀 Running prediction...
Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs

image 1/1 /content/datasets/Multi-hazard_Labeled_Dataset/images/test/speed_bump/Image00085.jpg: 384x640 1 Speed Bump, 50.1ms
Speed: 2.5ms preprocess, 50.1ms inference, 17.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/ultralytics/ultralytics/ultralytics/road_hazard_project/inference_results2
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [11]:
# Export the model to ONNX format
!yolo export \
model=road_hazard_project/exp_best_hyperparams/weights/best.pt \
format=onnx \
opset=12 \
simplify=True

Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'road_hazard_project/exp_best_hyperparams/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (5.2 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<2.0.0', 'onnxslim>=0.1.71', 'onnxruntime'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 185ms
Prepared 6 packages in 1.39s
Installed 6 packages in 298ms
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnx==1.20.1
 + onnxruntime==1.23.2
 + onnxslim==0.1.82

requirements: AutoUpdate success ✅ 2.4s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: st

In [12]:
# Export to standard TFLite
!yolo export \
model=road_hazard_project/exp_best_hyperparams/weights/best.pt \
format=tflite

Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'road_hazard_project/exp_best_hyperparams/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (5.2 MB)
E0000 00:00:1768202596.058424   34789 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768202596.125492   34789 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768202596.645999   34789 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768202596.646042   34789 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid link